# Рассмотрим для начала как отработает baseline (модель со значениями гиперпараметров по умолчанию).

In [1]:
# импортируем библиотеки
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [2]:
# убираем предупреждения с экрана
import warnings

warnings.filterwarnings('ignore')

In [3]:
# загружаем датасет Титаник
df = sns.load_dataset('titanic')

# удаляем неинформативные колонки
df.drop(columns=['alive', 'deck'], inplace=True)

# заполняем пропуски модами каждой из колонок
df = df.fillna(df.mode().iloc[0])

# разбиваем категориальные столбцы и удаляем один из классов,
# чтобы предобвратить мультиколлинеарность
df_label = pd.get_dummies(df, drop_first=True)

# выделяем X и y
X = df_label.drop('survived', axis=1)
y = df.survived

# разбиваем данные на train и test
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    stratify=y,
                                                    test_size=0.2,
                                                    random_state=42)

# стандартизируем данные
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

# обучаем модель логистической регрессии
clf = LogisticRegression()
clf.fit(X_train_std, y_train)

# получаем вероятности
y_train_proba = clf.predict_proba(X_train_std)
y_test_proba = clf.predict_proba(X_test_std)

# получаем значение метрики ROC-AUC
print(f'ROC-AUC train = {roc_auc_score(y_train, y_train_proba[:, 1]):.4f}')
print(f'ROC-AUC test = {roc_auc_score(y_test, y_test_proba[:, 1]):.4f}')

ROC-AUC train = 0.8691
ROC-AUC test = 0.8683


# Используем GridSearch при подборе

In [4]:
# словарь для перебора
parameters = {
    'max_iter': [1000],
    'C': [1, 10, 100],
    "solver": ["liblinear", "saga", "sag", "lbfgs"],
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    "random_state": [42],
    "class_weight" : ['balanced']
}

lr = LogisticRegression()
# подаем на вход модель, словарь с параметрами, установим, на какую метрику ориентироваться

clf = GridSearchCV(estimator=lr, # оценщик (лог рег)
                   param_grid=parameters,
                   scoring='roc_auc',  # метрика
                   cv=3,  # the cross-validation splitting strategy
                   verbose=0)  # отстутствие сообщений
clf.fit(X_train_std, y_train)

# получаем вероятности
y_train_grid = clf.predict_proba(X_train_std)
y_test_grid = clf.predict_proba(X_test_std)

# рассмотрим метрику ROC-AUC
print(f'ROC-AUC train = {roc_auc_score(y_train, y_train_grid[:, 1]):.4f}')
print(f'ROC-AUC test = {roc_auc_score(y_test, y_test_grid[:, 1]):.4f}')

ROC-AUC train = 0.8694
ROC-AUC test = 0.8687


# [sklearn.model_selection.GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

In [5]:
# посмотрим набор лучших параметров
clf.best_params_

{'C': 1,
 'class_weight': 'balanced',
 'max_iter': 1000,
 'penalty': 'l2',
 'random_state': 42,
 'solver': 'saga'}

In [6]:
# лучшая обученная модель
clf.best_estimator_

LogisticRegression(C=1, class_weight='balanced', max_iter=1000, random_state=42,
                   solver='saga')

# Используем RandomizedSearch при подборе

In [7]:
from sklearn.model_selection import RandomizedSearchCV

# создадим словарь с параметрами, которые хотим установить и перебрать
parameters = {
    'max_iter': [1000],
    'C': [1, 10, 100],
    "solver": ["liblinear", "saga", "sag", "lbfgs"],
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    "random_state": [42],
    "class_weight" : ['balanced']
}

lr_grid = LogisticRegression()
# подаем на вход модель, словарь с параметрами, установим, на какую метрику ориентироваться
clf = RandomizedSearchCV(estimator=lr_grid,
                         param_distributions=parameters,
                         random_state=42,
                         scoring='roc_auc',
                         cv=5,
                         verbose=0)
clf.fit(X_train_std, y_train)

# получение вероятностей
y_train_grid = clf.predict_proba(X_train_std)
y_proba_grid = clf.predict_proba(X_test_std)

print(f'ROC-AUC train = {roc_auc_score(y_train, y_train_grid[:,1]):.4f}')
print(f'ROC-AUC test = {roc_auc_score(y_test, y_proba_grid[:,1]):.4f}')

ROC-AUC train = 0.8694
ROC-AUC test = 0.8687


Также, если вы попробуете замерить время именно выполнения поиска для того и другого случая, то увидите, что RandomizedSearch работает быстрее GridSearch, но к сожалению, теряем в качестве.

# [sklearn.model_selection.RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)